# Linear Regression
This is my first Project on Linear Regression. In this Project I will try to predict the car fuel effeciency based on current data using my linear regression model.

There are several steps involved in it I have a decent amount of data which I got from https://www.fueleconomy.gov/feg/download.shtml which contains information related to cars and there mpg(miles per gallon).

The problem with this data is although it is excel sheet and csv file. There is still a small amount of incononsistencies in the data. 

Our first few steps will involve making our data uniform and properly labelled, so that our model can easily and effectively train on the model.

Right now I have data from 2000 till 2025. The first problem involves, how can I make the data in a uniform file format, because our data is in mixed files.

Since the majority of data files are in excel format. I will go ahead and make the other .csv files to .xlsx files.

In [1]:
import os
import pandas as pd

In [ ]:
# in the bellow cell, we will convert all the .csv files to .xlsx files in order to achieve uniform file format for our data
csv_directory = 'Data/'

for filename in os.listdir(csv_directory):
    if filename.endswith('.csv'):
        
        csv_file = os.path.join(csv_directory, filename)
        df = pd.read_csv(csv_file, on_bad_lines='skip')

        
        
        excel_file = os.path.join(csv_directory, filename.replace('.csv', '.xlsx'))

        df.to_excel(excel_file, index=False)
        print(f'Converted {filename} to {excel_file}')

        os.remove(csv_file)
        print(f'Deleted {csv_file}')